## Build your first NN from scratch
The code block below generates data for a regression problem.  The generator is similar to the generator for the classification problem that you've been working of for a while.  The main difference is how the labels are related to the attributes.  These labels are going require you to change several things.  For one thing, you may not want to use the sigmoid non-linearity.  Why?  For another, you'll have to define a loss function and work out the gradient equations.  

Follow the same steps that you've seen a couple of times now.  
1.  Define the inference model - network architecture
2.  Define the loss function
3.  Derive the gradient wrt to weights
4.  Use gradient descent to determine best parameters for your model.  
Hint: You won't use a sigmoid function. Instead, you'll just use $\hat{y} = w1 * x1 + w2 * x2$ instead. In the problem above, $\hat{y} = sigmoid(w1 * x1 + w2 * x2)$. Hence, the gradient will be different from the problem above. Loss function is the same: Mean Squared Error.

This is just the beginning.  Soon you'll be writing and solving much more complicated models than this - models having millions of free parameters instead of two.  But the principals and steps that you'll take to build and train those models will be exactly the same as for this simple one.  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

#numpy version of Data Generator 
sd = 0.2
nRows = 400
X = np.random.uniform(0.0, 1.0, (nRows, 2))
noise = np.random.normal(0.0, sd, (nRows,))
idx = X[:, 1] > X[:, 0] + noise
Y = np.zeros((nRows,))
Y[idx] = 1.0

#plot data
colorMap = {0.0: "blue", 1.0: "red"}
colors = [colorMap[c] for c in Y]
plt.scatter(X[:,0], X[:, 1], c=colors)
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

### Write a simple neural net using numpy
The simple neural net that you saw before was easy enough to write out each of the attributes and weights as separate variables, but the nets you'll be training later will have millions of parameters.  To deal with those, you'll use matrices and vectors to represent the input data and the weights that will be applies.  Matrix notation facilitates writing functions that will process all of the data in a single pass. 

Below is written some pretty awful script-type code. I challenge you to vectorize the algorithm in production-grade python, using the neural_network.py class we wrote in lecture. Or, even better, write your own class. Make the bridge between what I have for you here and what is in class to vectorize your code and make it efficient. 

In [ ]:
def npS(XX):
    return 1.0 / (1.0 + np.exp(-XX))

def npNeuralNet(XX, W):
    return npS(XX.dot(W))

def npLoss(XX, Y, W):
    return ((Y - npNeuralNet(XX, W)) ** 2).mean()

def npGradW(XX, Y, W):
    arg = XX.dot(W)
    terms = (-2.0 * (Y - npS(arg)) * npS(arg) * npS(-arg)).reshape(-1, 1)
    return (terms * XX).mean(axis=0).T

Below is a way of organizing your training pattern for gradient descent using a technique called [minibatching](https://machinelearningmastery.com/gentle-introduction-mini-batch-gradient-descent-configure-batch-size/). I recommend taking apart the code line by line, running it and seeing what it does.

In [ ]:
W = np.zeros((2,))
lr = 0.1
miniBatchSize = 50
nPasses = 300
loss = []
w_list = W

startEnd = zip(range(0, len(X), miniBatchSize), range(miniBatchSize, len(X) + 1, miniBatchSize))

for iPass in range(nPasses):
    for (s, e) in startEnd:
        x = X[s:e, ]
        y = Y[s:e]
        W = W - lr * npGradW(x, y, W)
        if not iPass % 5:
            loss.append(npLoss(x, y, W))
            print(iPass, npLoss(x, y, W), W)
            w_list = np.vstack((w_list, W))

In [ ]:
plt.plot(loss)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title("Loss over Time")
plt.show()

plt.scatter(w_list[:, 0], w_list[:, 1])
plt.xlabel("W1")
plt.ylabel("W2")
plt.title("Weights Updating Over Iterations")
plt.show()

plt.plot(w_list)
plt.xlabel("Iterations")
plt.ylabel("Weights: W1 and W2")
plt.title("Weights Updating Over Iterations")
plt.show()

## Exercises

1.  Twiddle parameters (batch size and learning rate) and see what settings give you the fastest convergence. What happens to the graphs when you change the parameters?  
2.  Build an iterator (class) that does the following
    -when you instantiate a member of the class, it generates the data set and initializes a counter
    -when you execute the member function "data(mbSize)" it generates the next mbSize points from the data set
    -when the "data" function reaches the end of the data, it starts over. 
3.  Modify the training code above by removing the "zip" and replacing the double loop over the data with a single loop that hits the iterator for as many steps as you choose. 
4.  Modify the data generator class so that when the "data" function runs out of data, it randomly scrambles the data rows before starting through it again.  Explain why do this.  